<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/mockup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip -q install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00


In [85]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')

In [86]:
data = data[data['claim_status'].isin(['Approve','Decline'])]

In [89]:
data[data['icd9']!='[]'].shape # จาก 50000 มีแค่ 104 rows

(104, 92)

In [99]:
print((data[data['icd9']!='[]']['chief_complaint'].iloc[3]))
print((data[data['icd9']!='[]']['is_procedure'].iloc[3]))
print((data[data['icd9']!='[]']['dx_free_text'].iloc[3]))
json.loads(data[data['icd9']!='[]']['diagnosis'].iloc[3])

ไข้ ไอ เจ็บคอ
Y
acute pharyngitis


[{'icd10': 'J02.9',
  'dx_name': 'Acute pharyngitis, unspecified',
  'dx_type': 'PP'}]

In [98]:
print((data[data['icd9']=='[]']['chief_complaint'].iloc[3]))
print((data[data['icd9']=='[]']['is_procedure'].iloc[3]))
print((data[data['icd9']=='[]']['icd9'].iloc[3]))
print((data[data['icd9']=='[]']['dx_free_text'].iloc[3]))
json.loads(data[data['icd9']=='[]']['diagnosis'].iloc[3])

ปวดท้องประจำเดือน
N
[]
Primary dysmenorrhoea


[{'icd10': 'N944', 'dx_name': 'Primary dysmenorrhoea', 'dx_type': 'PP'},
 {'icd10': 'N760', 'dx_name': 'Acute vaginitis', 'dx_type': 'OT'}]

In [6]:
icd10_df = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/icd10 - ICD-10 v.2019.csv')

In [66]:
data

Index(['id', 'seq', 'created_at', 'updated_at', 'deleted_at',
       'service_setting', 'illnes_type', 'policy_type', 'cid', 'cid_type',
       'claim_status', 'claim_status_desc', 'ref_id', 'transaction_uid',
       'transaction_no', 'is_follow_up', 'further_claim_id',
       'electronic_signature', 'accident_cause_over45_days', 'admit_date_time',
       'admit_date_time_txt', 'dsc_date_time', 'dsc_date_time_txt',
       'chief_complaint', 'is_procedure', 'icd9', 'dx_free_text', 'coma_score',
       'claim_from_motorcycle', 'visit_with_alcohol_or_drug',
       'alcohol_related', 'addiction_related', 'pregnant', 'total_bill_amount',
       'total_bill_amount_txt', 'patient', 'vital_sign', 'doctor',
       'order_item', 'billing', 'attach_doc_list', 'diagnosis', 'response',
       'check_eligible_id', 'insurance', 'insurance_id', 'hospital_code',
       'hospital_id', 'claim_no', 'discharge_date', 'discharge_date_txt',
       'member_id', 'further_claim', 'patient_registration_id',
    

## ข้อมูลโรค บิล

In [18]:
print(f'โรคทั้งหมดมี unique จำนวน', icd10_df['ICD-10 Code'].nunique(), 'โรค')
icd10_df[['ICD-10 Code','Descriptions (EN)']].head()

โรคทั้งหมดมี unique จำนวน 38385 โรค


,ICD-10 Code,Descriptions (EN)
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A009,"Cholera, unspecified"
3,A010,Typhoid fever
4,A011,Paratyphoid fever A


## ราคากลาง และ ช่วงราคาของค่าการรักษา แบ่งตามโรค

In [69]:
data.columns

Index(['id', 'seq', 'created_at', 'updated_at', 'deleted_at',
       'service_setting', 'illnes_type', 'policy_type', 'cid', 'cid_type',
       'claim_status', 'claim_status_desc', 'ref_id', 'transaction_uid',
       'transaction_no', 'is_follow_up', 'further_claim_id',
       'electronic_signature', 'accident_cause_over45_days', 'admit_date_time',
       'admit_date_time_txt', 'dsc_date_time', 'dsc_date_time_txt',
       'chief_complaint', 'is_procedure', 'icd9', 'dx_free_text', 'coma_score',
       'claim_from_motorcycle', 'visit_with_alcohol_or_drug',
       'alcohol_related', 'addiction_related', 'pregnant', 'total_bill_amount',
       'total_bill_amount_txt', 'patient', 'vital_sign', 'doctor',
       'order_item', 'billing', 'attach_doc_list', 'diagnosis', 'response',
       'check_eligible_id', 'insurance', 'insurance_id', 'hospital_code',
       'hospital_id', 'claim_no', 'discharge_date', 'discharge_date_txt',
       'member_id', 'further_claim', 'patient_registration_id',
    

In [70]:
import json
import pandas as pd

# สร้าง DataFrame จาก data
focus = pd.DataFrame(data)

# แปลงคอลัมน์ diagnosis และ order_item ให้เป็น JSON
focus['diagnosis'] = focus['diagnosis'].apply(lambda x: json.loads(x))
focus['order_item'] = focus['order_item'].apply(lambda x: json.loads(x))

all_billing_summary = []

# ดึงรายการรหัส ICD-10 ที่ไม่ซ้ำกัน
unique_icd10_codes = focus['diagnosis'].apply(lambda x: [d.get('icd10') for d in x]).explode().unique()

for icd10 in unique_icd10_codes:
    # กรองข้อมูลสำหรับ ICD-10 รหัสนั้น
    filtered_data = focus[
        focus['diagnosis'].apply(lambda x: any(d.get('icd10') == icd10 for d in x))
    ]

    # ตรวจหาค่า illness_type ที่พบบ่อยที่สุดสำหรับรหัส icd10 นั้น (หากมีหลายประเภท)
    illness_type = filtered_data['illnes_type'].mode().iloc[0] if not filtered_data['illnes_type'].mode().empty else None

    # สร้างรายการ billing
    billing_items = []
    for items in filtered_data['billing']:
        items = json.loads(items)
        for item in items:
            billing_items.append(item)

    billing_items_df = pd.DataFrame(billing_items)
    billing_items_df['billing_initial'] = billing_items_df['billing_initial'].astype(float)

    # คำนวณ summary ของ billing ตาม billing code และชื่อ
    billing_summary = billing_items_df.groupby(['simb_billing_code', 'local_billing_name']).agg(
        average_initial_price=('billing_initial', 'mean'),
        billing_count=('billing_initial', 'count'),
        min_initial_price=('billing_initial', 'min'),
        max_initial_price=('billing_initial', 'max')
    ).reset_index()

    # สร้างคอลัมน์ range
    billing_summary['range'] = billing_summary['min_initial_price'].astype(str) + '-' + billing_summary['max_initial_price'].astype(str)

    # เพิ่มคอลัมน์ icd10 และ illness_type
    billing_summary['icd10'] = icd10
    billing_summary['illness_type'] = illness_type

    # เพิ่มข้อมูล billing_summary เข้าในรายการ all_billing_summary
    all_billing_summary.append(billing_summary)

# รวมข้อมูลทั้งหมดใน all_billing_summary เข้าด้วยกัน
final_summary = pd.concat(all_billing_summary, ignore_index=True)

# แสดงตารางผลลัพธ์
display(final_summary)

,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,range,icd10,illness_type
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,939.212121,33,0.0,3581.0,0.0-3581.0,S711,ACC
1,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,241.041667,24,45.0,650.0,45.0-650.0,S711,ACC
2,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,228.181818,11,100.0,300.0,100.0-300.0,S711,ACC
3,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,170.833333,24,50.0,310.0,50.0-310.0,S711,ACC
4,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,447.095238,21,34.0,1305.0,34.0-1305.0,S711,ACC
...,...,...,...,...,...,...,...,...,...
14091,1.2.1(1),First Outpatient care before admission ค่าแพท...,350.000000,1,350.0,350.0,350.0-350.0,T17,ACC
14092,1.2.2,Physician Procedures ค่าแพทย์ศัลยกรรม หัตถกา...,1000.000000,1,1000.0,1000.0,1000.0-1000.0,T17,ACC
14093,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,735.000000,1,735.0,735.0,735.0-735.0,K29.1,ILL
14094,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,120.000000,1,120.0,120.0,120.0-120.0,K29.1,ILL


In [73]:
final_summary.to_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/range_billing_by_icd10.csv', index=False)

## **mockup data**
>

> **diagnosis + billing**
1. **เลือกโรค (ICD-10)**: สุ่มโรคจากตาราง icd10_df เพื่อใส่ลงในข้อมูล mockup ของแต่ละแถว โดยในแต่ละโรคจะมีรายการ billing ที่เหมาะสมตามตาราง final_summary
2. **สร้างรายการ billing**: ใช้ข้อมูลใน final_summary เพื่อกำหนดช่วงราคาของแต่ละรายการใน billing โดยสุ่มราคาในช่วงที่กำหนด (range) สำหรับรายการนั้นๆ

In [75]:
# Sample data for each field
hospital_codes = [11787, 40765, 11583, 11558, 11768, 23781, 41829, 11864, 14683,
                  21673, 11720, 11771, 14591, 11543, 41345, 24258, 11963, 24257,
                  13766, 11645, 14354, 14550, 11766, 23924, 11878, 12026, 41556,
                  12188, 12226, 12159, 13779, 41674, 14641, 14534, 12139, 11765,
                  28875, 42025, 41590, 23875, 11919, 11730, 11537, 10814, 41569,
                  11845, 11708, 11644, 11648, 10686, 44109, 10827, 10734, 10663,
                  10679, 42167, 11874, 42770, 10821, 44353, 42015, 11037, 11656,
                  10695, 12230, 42141, 10849, 11359, 11621, 12212, 13778, 11920,
                  10674, 10723, 10671, 10998, 10822, 10667, 11360, 10832]

In [101]:
import pandas as pd
import random
from faker import Faker
import json

# Initial setup
fake = Faker(['en_US', 'th_TH'])
num_rows = 5000  # จำนวนแถวที่ต้องการสร้างข้อมูล mockup

# Filter ICD-10 codes based on final_summary
icd10_codes_in_summary = final_summary['icd10'].unique()
diagnosis_samples = [
    {'icd10': row['ICD-10 Code'], 'dx_name': row['Descriptions (EN)']}
    for _, row in icd10_df.iterrows()
    if row['ICD-10 Code'] in icd10_codes_in_summary
]

# Generate mock data
data = []
for _ in range(num_rows):
    # Basic patient and doctor info
    hospital_code = random.choice(hospital_codes)
    cid = fake.unique.random_number(digits=13)
    patient = {
        'hn': f"{random.randint(10, 99)}-{random.randint(10000, 99999)}",
        'dob': fake.date_of_birth().isoformat(),
        'gender': random.choice(['M', 'F']),
        'last_name': fake.last_name(),
        'first_name': fake.first_name(),
        'title_name': random.choice(['นาย', 'นาง', 'นางสาว']),
        'patient_registration_id': fake.uuid4()
    }
    vital_sign = [{
        'heart_rate': str(random.randint(60, 100)),
        'pain_score': str(random.randint(0, 10)),
        'systolic_bp': f"{random.uniform(90, 180):.2f}",
        'temperature': f"{random.uniform(36.0, 37.5):.2f}",
        'diastolic_bp': f"{random.uniform(60, 120):.2f}",
        'respiratory_rate': str(random.randint(12, 20)),
        'oxygen_saturation': f"{random.uniform(95, 100):.2f}",
        'vital_sign_entry_datetime': fake.date_time_this_year().strftime("%Y-%m-%d %H:%M")
    }]
    doctor = [{
        'role': 'OWNER',
        'title': 'นพ.',
        'license': str(random.randint(10000, 99999)),
        'lastname': fake.last_name(),
        'firstname': fake.first_name()
    }]

    # Select only one random diagnosis that exists in final_summary
    diagnosis = [random.choice(diagnosis_samples)]
    diag = diagnosis[0]

    # Retrieve illness_type for the selected diagnosis ICD-10 from final_summary
    illness_type = final_summary.loc[final_summary['icd10'] == diag['icd10'], 'illness_type'].values[0]

    # Generate billing items for the single diagnosis
    billing = []

    # Filter `final_summary` by the selected diagnosis ICD-10 code
    possible_billing_items = final_summary[final_summary['icd10'] == diag['icd10']]

    for _, billing_row in possible_billing_items.iterrows():
        billing_initial = random.uniform(billing_row['min_initial_price'], billing_row['max_initial_price'])
        billing_discount = random.uniform(0, billing_initial * 0.2)  # Random discount up to 20%
        billing_net_amount = billing_initial - billing_discount

        billing.append({
            'billing_initial': f"{billing_initial:.2f}",
            'billing_discount': f"{billing_discount:.2f}",
            'simb_billing_code': billing_row['simb_billing_code'],
            'billing_net_amount': f"{billing_net_amount:.2f}",
            'local_billing_code': billing_row['simb_billing_code'],
            'local_billing_name': billing_row['local_billing_name']
        })

    # Calculate total_bill_amount by summing billing_net_amount in billing
    total_bill_amount = sum(float(item['billing_net_amount']) for item in billing)

    # Append the row with all information, including illness_type and total_bill_amount
    data.append({
        'hospital_code': hospital_code,
        'cid': cid,
        'patient': patient,
        'diagnosis': diagnosis,
        'vital_sign': vital_sign,
        'doctor': doctor,
        'billing': billing,
        'illness_type': illness_type,  # Add illness_type here
        'total_bill_amount': f"{total_bill_amount:.2f}"  # Add total_bill_amount here
    })


In [102]:
# Create DataFrame and display
mock2 = pd.DataFrame(data)
display(mock2)

,hospital_code,cid,patient,diagnosis,vital_sign,doctor,billing,illness_type,total_bill_amount
0,12139,1802644268289,"{'hn': '22-18744', 'dob': '1966-11-13', 'gende...","[{'icd10': 'C400', 'dx_name': 'Malignant neopl...","[{'heart_rate': '84', 'pain_score': '2', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '408.00', 'billing_discou...",ACC,1691.51
1,12159,7182954973624,"{'hn': '40-44063', 'dob': '1973-05-11', 'gende...","[{'icd10': 'Z138', 'dx_name': 'Special screeni...","[{'heart_rate': '76', 'pain_score': '4', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '400.00', 'billing_discou...",ILL,385.83
2,10674,5582006127104,"{'hn': '26-28895', 'dob': '1956-11-20', 'gende...","[{'icd10': 'W222', 'dx_name': 'Striking agains...","[{'heart_rate': '81', 'pain_score': '4', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '1060.00', 'billing_disco...",ACC,2916.26
3,41590,6732625613068,"{'hn': '99-37569', 'dob': '1921-06-28', 'gende...","[{'icd10': 'S6088', 'dx_name': 'Other superfic...","[{'heart_rate': '69', 'pain_score': '3', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '1596.15', 'billing_disco...",ACC,5505.57
4,21673,6206786603046,"{'hn': '69-86004', 'dob': '1950-08-06', 'gende...","[{'icd10': 'S140', 'dx_name': 'Concussion and ...","[{'heart_rate': '86', 'pain_score': '6', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '1459.00', 'billing_disco...",ACC,1855.36
...,...,...,...,...,...,...,...,...,...
4995,11720,3543038219951,"{'hn': '56-46166', 'dob': '1945-12-14', 'gende...","[{'icd10': 'G404', 'dx_name': 'Other generaliz...","[{'heart_rate': '62', 'pain_score': '0', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '775.00', 'billing_discou...",ILL,969.68
4996,11878,9988779469431,"{'hn': '43-74084', 'dob': '1921-03-13', 'gende...","[{'icd10': 'J312', 'dx_name': 'Chronic pharyng...","[{'heart_rate': '80', 'pain_score': '2', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '198.63', 'billing_discou...",ILL,3475.21
4997,11845,4475869784578,"{'hn': '29-48366', 'dob': '1978-06-14', 'gende...","[{'icd10': 'S52109', 'dx_name': 'Closed Fractu...","[{'heart_rate': '82', 'pain_score': '3', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '710.00', 'billing_discou...",ACC,6172.55
4998,11919,6036241986836,"{'hn': '65-48274', 'dob': '1952-11-03', 'gende...","[{'icd10': 'N762', 'dx_name': 'Acute vulvitis'}]","[{'heart_rate': '67', 'pain_score': '9', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '480.00', 'billing_discou...",ILL,1118.12


In [118]:
mock2.to_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/mock1.csv')

#### check 1

In [119]:
filtered_data = final_summary[final_summary['icd10'] == 'N762']
display(filtered_data)
print(filtered_data['average_initial_price'].sum())

,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,range,icd10,illness_type
13359,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,480.0,1,480.0,480.0,480.0-480.0,N762,ILL
13360,1.2.1(1),First Outpatient care before admission ค่าแพท...,600.0,1,600.0,600.0,600.0-600.0,N762,ILL
13361,1.2.2,Physician Procedures ค่าแพทย์ศัลยกรรม หัตถกา...,120.0,1,120.0,120.0,120.0-120.0,N762,ILL


1200.0


In [120]:
print(mock2['total_bill_amount'].iloc[-2])
print(mock2['diagnosis'].iloc[-2], '\n')
mock2['billing'].iloc[-2]

1118.12
[{'icd10': 'N762', 'dx_name': 'Acute vulvitis'}] 



[{'billing_initial': '480.00',
  'billing_discount': '62.39',
  'simb_billing_code': '1.1.1(3)',
  'billing_net_amount': '417.61',
  'local_billing_code': '1.1.1(3)',
  'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'},
 {'billing_initial': '600.00',
  'billing_discount': '5.13',
  'simb_billing_code': '1.2.1(1)',
  'billing_net_amount': '594.87',
  'local_billing_code': '1.2.1(1)',
  'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'},
 {'billing_initial': '120.00',
  'billing_discount': '14.36',
  'simb_billing_code': '1.2.2',
  'billing_net_amount': '105.64',
  'local_billing_code': '1.2.2',
  'local_billing_name': 'Physician Procedures  ค่าแพทย์ศัลยกรรม  หัตถการผ่าตัด'}]

#### check 2

In [117]:
filtered_data = final_summary[final_summary['icd10'] == 'J312']
display(filtered_data)
print(filtered_data['average_initial_price'].sum())

,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,range,icd10,illness_type
8597,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,441.2,5,65.0,833.0,65.0-833.0,J312,ILL
8598,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,42.5,2,40.0,45.0,40.0-45.0,J312,ILL
8599,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,224.2,5,81.0,400.0,81.0-400.0,J312,ILL
8600,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,14.0,1,14.0,14.0,14.0-14.0,J312,ILL
8601,1.2.1(1),First Outpatient care before admission ค่าแพท...,300.0,2,300.0,300.0,300.0-300.0,J312,ILL
8602,1.2.1(11),Special Diagnositics Physician Service ค่าตรว...,325.0,2,300.0,350.0,300.0-350.0,J312,ILL
8603,1.2.1(4),Emergency Outpatient Care within 24 hr ตรวจรัก...,800.0,1,800.0,800.0,800.0-800.0,J312,ILL
8604,1.2.2,Physician Procedures ค่าแพทย์ศัลยกรรม หัตถกา...,1800.0,1,1800.0,1800.0,1800.0-1800.0,J312,ILL
8605,1.2.3(3),Nurse - Midwife Professional Fees ค่าพยาบาลห้...,275.0,1,275.0,275.0,275.0-275.0,J312,ILL


4221.9


In [112]:
print(mock2['total_bill_amount'].iloc[-4])
print(mock2['diagnosis'].iloc[-4], '\n')
mock2['billing'].iloc[-4]

3475.21
[{'icd10': 'J312', 'dx_name': 'Chronic pharyngitis'}] 



[{'billing_initial': '198.63',
  'billing_discount': '5.02',
  'simb_billing_code': '1.1.1(3)',
  'billing_net_amount': '193.61',
  'local_billing_code': '1.1.1(3)',
  'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'},
 {'billing_initial': '40.42',
  'billing_discount': '3.57',
  'simb_billing_code': '1.1.12',
  'billing_net_amount': '36.85',
  'local_billing_code': '1.1.12',
  'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'},
 {'billing_initial': '164.16',
  'billing_discount': '18.64',
  'simb_billing_code': '1.1.14(2)',
  'billing_net_amount': '145.52',
  'local_billing_code': '1.1.14(2)',
  'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'},
 {'billing_initial': '14.00',
  'billing_discount': '2.38',
  'simb_billing_code': '1.1.2(1)',
  'billing_net_amount': '11.62',
  'local_billing_code': '1.1.2(1)',
  'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'},
 {'billing_initial': '300.00',
  'billing_discount': '14.24',